In [5]:

import math
import pandas as pd


weight_kg = 7.26   # 16 lbs ≈ 7.26 kg
unit_value_usd = 100.0  # estimated COGS for holding cost calc
holding_rate = 0.15     # 15% annual cost of capital
days_in_year = 365

# Air freight assumptions
air_rate_usd_per_kg = 7.0   # all-in (base + fuel + security); oversized surcharges modeled below
air_oversize_surcharge = 200.0  # per unit, due to 1.6 m length
p_air = 0.01   # probability of having to expedite by air due to delays

# Ocean freight (China) assumptions
ocean_rate_40HC_normal = 3500.0   # USD per 40' HC container (normal)
ocean_rate_40HC_crisis_mult = 3.0 # crisis multiplier (e.g., Red Sea / port disruptions)
units_per_container = 150         # due to size + packaging efficiency
us_port_drayage_per_container = 2000.0  # trucking & handling from port to plant per container (US side)
cn_ocean_days = 30                # normal ocean transit (port-to-port + handling buffer)
cn_extra_delay_days = 10          # additional delays (port congestion / customs)
cn_crisis_prob = 0.01             # probability of crisis conditions on a given period

# Mexico trucking assumptions
mx_distance_miles = 1500          # average plant-to-plant distance (e.g., MX plant to TX/NV/CA)
truck_cost_per_mile = 2.50        # full truckload line-haul cost per mile
mx_units_per_truck = 400          # loading efficiency
mx_border_fee_per_truck = 500.0   # brokerage + border fees
mx_border_delay_days = 2          # typical added time at border
mx_truck_delay_prob = 0.30        # probability of notable delay that triggers air (rare) or expediting domestically
mx_p_air = 0.01                   # small probability of switching a small batch to air (e.g., service recovery)

# US domestic trucking assumptions
us_distance_miles = 2000
us_units_per_truck = 400
us_truck_delay_days = 1
us_p_air = 0.01  # very rare expedited air domestically for this part


def holding_cost_per_unit(extra_days, unit_value=unit_value_usd, rate=holding_rate):
    return unit_value * rate * (extra_days / days_in_year)

def air_cost_per_unit(weight=weight_kg, rate_per_kg=air_rate_usd_per_kg, oversize=air_oversize_surcharge):
    return weight * rate_per_kg + oversize

def expected_air_cost(p_air, air_unit_cost):
    return p_air * air_unit_cost

def china_transport_cost():
    # Base ocean per unit (normal)
    ocean_normal_per_unit = ocean_rate_40HC_normal / units_per_container
    drayage_per_unit = us_port_drayage_per_container / units_per_container
    
    # Crisis expected multiplier
    expected_multiplier = (1 - cn_crisis_prob) * 1.0 + cn_crisis_prob * ocean_rate_40HC_crisis_mult
    expected_ocean_per_unit = (ocean_rate_40HC_normal * expected_multiplier) / units_per_container + drayage_per_unit
    
    # Holding cost due to time in transit
    exp_days = cn_ocean_days + cn_extra_delay_days * cn_crisis_prob  # expected extra days
    holding = holding_cost_per_unit(exp_days)
    
    # Expected expedited air (if schedule risk forces small batch air)
    air_unit = air_cost_per_unit()
    exp_air = expected_air_cost(p_air, air_unit)
    
    total = expected_ocean_per_unit + holding + exp_air
    detail = {
        "Base_ocean_per_unit_normal": round(ocean_normal_per_unit, 2),
        "Drayage_per_unit": round(drayage_per_unit, 2),
        "Expected_ocean_per_unit": round(expected_ocean_per_unit, 2),
        "Holding_cost": round(holding, 2),
        "Expected_air_cost": round(exp_air, 2),
        "Total_transport_per_unit": round(total, 2),
    }
    return detail

def mexico_transport_cost():
    # Base trucking per unit
    truck_total = truck_cost_per_mile * mx_distance_miles
    per_truck_all_in = truck_total + mx_border_fee_per_truck
    base_per_unit = per_truck_all_in / mx_units_per_truck
    
    # Holding due to border delay
    holding = holding_cost_per_unit(mx_border_delay_days)
    
    # Rare expedited air
    air_unit = air_cost_per_unit()
    exp_air = expected_air_cost(mx_p_air, air_unit)
    
    total = base_per_unit + holding + exp_air
    detail = {
        "Base_truck_per_unit": round(base_per_unit, 2),
        "Holding_cost": round(holding, 2),
        "Expected_air_cost": round(exp_air, 2),
        "Total_transport_per_unit": round(total, 2),
    }
    return detail

def us_transport_cost():
    # Base trucking per unit
    truck_total = truck_cost_per_mile * us_distance_miles
    base_per_unit = truck_total / us_units_per_truck
    
    # Minimal delay holding
    holding = holding_cost_per_unit(us_truck_delay_days)
    
    # Very rare expedited air
    air_unit = air_cost_per_unit()
    exp_air = expected_air_cost(us_p_air, air_unit)
    
    total = base_per_unit + holding + exp_air
    detail = {
        "Base_truck_per_unit": round(base_per_unit, 2),
        "Holding_cost": round(holding, 2),
        "Expected_air_cost": round(exp_air, 2),
        "Total_transport_per_unit": round(total, 2),
    }
    return detail

cn = china_transport_cost()
mx = mexico_transport_cost()
us = us_transport_cost()

df = pd.DataFrame.from_records([
    {"Route": "China → US (Ocean + Drayage)", **cn},
    {"Route": "Mexico → US (Truck + Border)", **mx},
    {"Route": "US Domestic (Truck)", **us},
])
print(df.to_string(index=False))



                       Route  Base_ocean_per_unit_normal  Drayage_per_unit  Expected_ocean_per_unit  Holding_cost  Expected_air_cost  Total_transport_per_unit  Base_truck_per_unit
China → US (Ocean + Drayage)                       23.33             13.33                    37.13          1.24               2.51                     40.88                  NaN
Mexico → US (Truck + Border)                         NaN               NaN                      NaN          0.08               2.51                     13.22                10.62
         US Domestic (Truck)                         NaN               NaN                      NaN          0.04               2.51                     15.05                12.50
